In [4]:
import requests
from sqlalchemy import create_engine
from urllib.parse import quote_plus
from langchain.globals import set_debug
from langchain_community.utilities import SQLDatabase
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import AIMessage, HumanMessage
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain.agents import tool
from langchain.agents import AgentExecutor
import boto3
from langchain_community.chat_message_histories import (
    DynamoDBChatMessageHistory,
)

from langchain_community.agent_toolkits import SQLDatabaseToolkit
import os

# Get the service resource.
dynamodb = boto3.resource("dynamodb")

AWS_REGION = "us-east-1" # Change me
SCHEMA_NAME = "wnba_db" # Athena calls this a database
S3_STAGING_DIR = "s3://wnbadata/" # Change me

connect_str = "awsathena+rest://athena.{region_name}.amazonaws.com:443/{schema_name}?s3_staging_dir={s3_staging_dir}"

engine = create_engine(connect_str.format(
        region_name=AWS_REGION,
        schema_name=SCHEMA_NAME,
        s3_staging_dir=quote_plus(S3_STAGING_DIR)
))

db = SQLDatabase(engine, ignore_tables = ["wnba_player_box_new"])
#db = SQLDatabase(engine)
schema = db.get_table_info()

os.environ["OPENAI_API_KEY"] = "sk-proj-Jz56EPt2tMFrDJtB1Q2dT3BlbkFJBOkimFUK14hExlzvsvJF"

llm = ChatOpenAI( model_name= "gpt-4", temperature= 0)

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

tools = toolkit.get_tools()


llm_with_tools = llm.bind_tools(tools)

prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        """You are an agent designed to interact with a SQL database.
        Given an input question, create a syntactically correct SQLite query to run, then look at the results of the query and return the answer.
        Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 5 results.
        You can order the results by a relevant column to return the most interesting examples in the database.
        Never query for all the columns from a specific table, only ask for the relevant columns given the question.
        You have access to tools for interacting with the database.
        Only use the below tools. Only use the information returned by the below tools to construct your final answer.
        You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.
        
        DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.
        
        To start you should ALWAYS look at the tables in the database to see what you can query.
        Do NOT skip this step.
        Then you should query the schema of the most relevant tables.""",
    ),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
        "chat_history": lambda x: x["chat_history"],
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)
agent_executor = AgentExecutor(agent = agent, tools = tools, verbose = False, return_intermediate_steps = True)

#chat_history = []

my_key = {
    "SessionId": "session_id::0",
    "UserID": "0001",
}

chat_history = DynamoDBChatMessageHistory(
    table_name="Chat_Table",
    session_id="0",
    key=my_key,
)

input_text = input('>>> ')
while input_text.lower() != 'bye':
    if input_text:
        print("loading response")
        response = agent_executor.invoke({
            'input': input_text,
            'chat_history': chat_history.messages,
        })
        chat_history.add_user_message(input_text)
        chat_history.add_ai_message(response["output"])
        '''
        chat_history.extend([
            HumanMessage(content=input_text),
            AIMessage(content=response["output"]),
        ])
        '''
        print(response['output'])
    input_text = input('>>> ')

/var/folders/1c/ybx_8cds1ybgdg3z19ptmmk00000gn/T/ipykernel_56807/1075996143.py:33: SADeprecationWarning: The dbapi() classmethod on dialect classes has been renamed to import_dbapi().  Implement an import_dbapi() classmethod directly on class <class 'pyathena.sqlalchemy.rest.AthenaRestDialect'> to remove this warning; the old .dbapi() classmethod may be maintained for backwards compatibility.
  engine = create_engine(connect_str.format(


>>>  Who played the most games in 2021?


loading response
The players who played the most games in 2021 are:

1. Skylar Diggins-Smith with 44 games
2. Shey Peddy with 43 games
3. Megan Walker with 43 games
4. Kahleah Copper with 43 games
5. Courtney Vandersloot with 43 games


>>>  Among these players, who scored the most?


loading response
Among these players, the one who scored the most in 2021 is:

1. Skylar Diggins-Smith with 577 points
2. Kahleah Copper with 545 points
3. Courtney Vandersloot with 444 points
4. Shey Peddy with 129 points
5. Megan Walker with 96 points


>>>  bye
